In [1]:
#!pip install numpy
#!pip install nltk
#!pip install tensorflow
#!pip install tflearn

In [2]:
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer=LancasterStemmer()
import numpy
import time
import tflearn
import tensorflow
import random
import pickle
import json
#with open('intents.json') as file:
#    data=json.load(file)
with open('intents_ZH-TW.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
'''
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
'''
words=[]
labels=[]
docs_x=[]
docs_y=[]
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        wrds=nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append (intent["tag"])


In [4]:
words=[stemmer.stem(w.lower()) for w in words if w !="?"]
words=sorted(list(set(words)))
labels=sorted(labels)

In [5]:
training=[]
output=[]
out_empty=[0 for _ in range(len(labels))]
for x,doc in enumerate(docs_x):
    bag=[]
    wrds=[stemmer.stem(w.lower()) for w in doc]
    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)
    output_row=out_empty[:]
    output_row[labels.index(docs_y[x])]=1

    training.append(bag)
    output.append(output_row)

In [6]:
training=numpy.array(training)
output=numpy.array(output)

In [7]:
import tensorflow as tf
from tensorflow.keras import backend as K
import tflearn

K.clear_session()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
model.fit(training,output,n_epoch=1500,batch_size=8,show_metric=True)
model.save("model.tflearn")

Training Step: 17999  | total loss: 2.13961 | time: 0.028s
| Adam | epoch: 1500 | loss: 2.13961 - acc: 0.9000 -- iter: 88/94
Training Step: 18000  | total loss: 1.92582 | time: 0.030s
| Adam | epoch: 1500 | loss: 1.92582 - acc: 0.9100 -- iter: 94/94
--
INFO:tensorflow:C:\Users\user\BotTrial\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [10]:
import speech_recognition as sr
def listenTo():
    r = sr.Recognizer()

    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        audio = r.listen(source)

    return r.recognize_google(audio, language='zh-TW')
#Course 88
import tempfile
from gtts import gTTS
from pygame import mixer
mixer.init()

def speak(sentence):
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts = gTTS(text=sentence, lang='zh-tw')
        tts.save("{}.mp3".format(fp.name))
        mixer.music.load('{}.mp3'.format(fp.name))
        mixer.music.play()

pygame 2.5.2 (SDL 2.28.3, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [11]:
def chat():
    print("開始與您的機器人對話（輸入 結束 以停止）：")
    speak("開始與您的機器人對話（輸入 結束 以停止）：")
    time.sleep(5)
    while True:
        try:
            inp=listenTo()
            print("You:",inp)
            time.sleep(2)
            if inp=="結束":
                break
            results=model.predict([bag_of_words(inp,words)])
            results_index=numpy.argmax(results)
            tag=labels[results_index]

            for tg in data["intents"]:
                if tg["tag"]==tag:
                    responses=tg["responses"]
            rd=random.choice(responses)
            speak(rd)
            print(rd)
            time.sleep(5)
        except sr.UnknownValueError:
            print("我無法理解音訊")
        except sr.RequestError as e:
            print("無法從 Google 語音識別服務請求結果; {0}".format(e))


In [12]:
chat()

開始與您的機器人對話（輸入 結束 以停止）：
我無法理解音訊
You: 請給我詳細的解決方法結束
在尋找協助時，可以嘗試聯繫當地的心理健康機構，他們會有相應的資源和專業人員。
我無法理解音訊
我無法理解音訊
You: 運動能解決嗎
運動對於改善心理健康有一定的幫助，它有助於釋放身體中的一些正面化學物質。你可以嘗試一些輕鬆的運動，看看對你的情緒有何影響。
我無法理解音訊
You: 我覺得你這些用處不大
有些人選擇與社交工作者、心理治療師或心理輔導師合作。他們都是可以提供支持和指導的專業人士。
我無法理解音訊
You: 結束
